# Importacion de dataset y librerias necesarias.


In [1]:
!pip install datasets
!pip install evaluate
!pip install transformers==4.28.0 datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/81.4 kB ? eta -:--:--

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 3.0 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/7.0 MB ? eta -:--:--

     ━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.3/7.0 MB 9.4 MB/s eta 0:00:01

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 6.9/7.0 MB 73.4 MB/s eta 0:00:01

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 51.9 MB/s eta 0:00:00


  Attempting uninstall: transformers


    Found existing installation: transformers 4.30.1


    Uninstalling transformers-4.30.1:


      Successfully uninstalled transformers-4.30.1


In [2]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/227.6 kB ? eta -:--:--

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 225.3/227.6 kB 7.8 MB/s eta 0:00:01

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 2.5 MB/s eta 0:00:00


  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.12.0


    Uninstalling accelerate-0.12.0:


      Successfully uninstalled accelerate-0.12.0


In [3]:
!apt install git-lfs
!git config --global user.email "jyarac@unal.edu.co"
!git config --global user.name "jyarac"



git-lfs is already the newest version (3.0.2-1ubuntu0.2).


0 upgraded, 0 newly installed, 0 to remove and 67 not upgraded.


In [4]:
from datasets import load_dataset, Dataset, Features, ClassLabel, Sequence, Value, DatasetDict
import pandas as pd

data = pd.read_csv('../input/restmex/Rest_Mex_Sentiment_Analysis_Train.tsv', sep='\t', index_col=False)
data_test = pd.read_csv('../input/restmex/Rest_Mex_Sentiment_Analysis_Test.tsv', sep='\t', index_col=False)


# Preprocesamiento


In [5]:
data_test.dropna(subset=['Review'], inplace=True)
data.dropna(subset=['Review'], inplace=True)

In [6]:
data_test.drop(["ID", "Unnamed: 2"], axis=1, inplace= True)
data.drop(["Unnamed: 2", "Unnamed: 3"], axis=1, inplace= True)


In [7]:
from sklearn.model_selection import train_test_split
a,b = train_test_split(data, test_size=0.30, stratify = data['Class'])


In [8]:
from datasets import Dataset, concatenate_datasets, DatasetDict

ds_train = Dataset.from_pandas(a)
ds_test = Dataset.from_pandas(b)
ds_train = ds_train.remove_columns('__index_level_0__')
ds_test = ds_test.remove_columns('__index_level_0__')
ds_dict = {'train' : ds_train, 'test':ds_test}
dataset = DatasetDict(ds_dict)
dataset = dataset.rename_column("Class", "labels")
dataset

DatasetDict({
    train: Dataset({
        features: ['Review', 'labels'],
        num_rows: 12250
    })
    test: Dataset({
        features: ['Review', 'labels'],
        num_rows: 5250
    })
})

In [9]:
def polarity_init(example):
  example["labels"] = example["labels"] - 1
  return example

dataset = dataset.map(polarity_init)

dataset.set_format("pandas")
df = dataset["train"][:]
print(df["labels"].value_counts())
dataset.reset_format()

  0%|          | 0/12250 [00:00<?, ?ex/s]

  0%|          | 0/5250 [00:00<?, ?ex/s]

4    3675
3    3062
2    2450
1    1838
0    1225
Name: labels, dtype: int64


#tokenizacion


In [10]:
from transformers import AutoTokenizer
model_checkpoint= "nlptown/bert-base-multilingual-uncased-sentiment"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


In [11]:
def tokenize_reviews(examples):
    return tokenizer(examples["Review"], truncation=True)

columns = dataset["train"].column_names
columns.remove("labels")
encoded_dataset = dataset.map(tokenize_reviews, batched=True, remove_columns=columns)
encoded_dataset

  0%|          | 0/13 [00:00<?, ?ba/s]

  0%|          | 0/6 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 12250
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 5250
    })
})

#Cargar modelo y metricas


In [12]:
from transformers import AutoModelForSequenceClassification

num_labels = 5
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [13]:
from datasets import load_metric

metric = load_metric("f1")
metric

Metric(name: "f1", features: {'predictions': Value(dtype='int32', id=None), 'references': Value(dtype='int32', id=None)}, usage: """
Args:
    predictions: Predicted labels, as returned by a model.
    references: Ground truth labels.
    labels: The set of labels to include when average != 'binary', and
        their order if average is None. Labels present in the data can
        be excluded, for example to calculate a multiclass average ignoring
        a majority negative class, while labels not present in the data will
        result in 0 components in a macro average. For multilabel targets,
        labels are column indices. By default, all labels in y_true and
        y_pred are used in sorted order.
    average: This parameter is required for multiclass/multilabel targets.
        If None, the scores for each class are returned. Otherwise, this
        determines the type of averaging performed on the data:
            binary: Only report results for the class specified by pos

In [14]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels, average="weighted")

# Fine tuning


In [ ]:
!huggingface-cli login

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from transformers import TrainingArguments

model_name = model_checkpoint.split("/")[-1]

batch_size = 16
num_train_epochs=4
train_dataset = encoded_dataset["train"]
logging_steps = len(train_dataset) // (2 * batch_size * num_train_epochs)

training_args = TrainingArguments(
    output_dir="results",
    num_train_epochs=num_train_epochs,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_steps=logging_steps,
    push_to_hub=True,
    push_to_hub_model_id=f"{model_name}-MeIA"
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=encoded_dataset["test"],
    tokenizer=tokenizer
)

In [ ]:
trainer.train()

In [ ]:
eval_results = trainer.evaluate()
print(eval_results)

# SADASD


In [ ]:
trainer.push_to_hub()

In [ ]:
from transformers import pipeline
pi = pipeline("text-classification", model="jyarac/bert-base-multilingual-uncased-sentiment-MeIA", max_length=512, truncation=True)

In [ ]:
#Abrir archivo de texto en modo escritura para almacenar los resultados para el reto
with open('optimizadoresDePalabras-3.txt', 'w') as f:
  for i in range(len(data_test)):
      res = pi(data_test.iloc[i]['Review'])
      label = res[0]['label']
      #Establecer la etiqueta correspondiente
      if label == 'LABEL_0':
        label = "1"
      elif label == 'LABEL_1':
        label = "2"
      elif label == 'LABEL_2':
        label = "3"
      elif label == 'LABEL_3':
        label = "4"
      elif label == 'LABEL_4':
        label = "5"
      #Escribir predicción del sentimiento en el archivo de texto con el formaro correspondiente
      f.write(str(i)+"\t"+label+"\n")
    
    #Imprimir en pantalla lo que se escribio en el archivo txt
      #Descomente si solo quiere probar este bloque de codigo para las primeras tres instancias de pruebas
print("ready")